In [17]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

# Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

column_renaming_dict = {'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'}

In [18]:
static_traces = []
static_traces_times = []


# read in Gordian scatter map
dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
#dfv = dfv.loc[dfv['extinction'].between(0.0003, 1)]
dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
ds_index = 2
scatter_edenhofer = go.Scatter3d(
    x=dfv['X'].values[::ds_index],
    y=dfv['Y'].values[::ds_index],
    z=dfv['Z'].values[::ds_index],
    mode='markers',
    marker=dict(size=5.,
                color='gray',
                # color=dfv['extinction'].values[::ds_index],
                # colorscale='Gray_r',
                cmin = 0.0007,
                cmax = 0.005,
                symbol='circle',
                opacity=.01),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0


hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
static_traces.append(scatter_lb)
static_traces_times.append([0.]) # only show at t=0

In [19]:
df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_catalog_oct_13.csv')
df_hunt_full = df_hunt_full.rename(columns={'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'})

df_hunt_good = df_hunt_full.loc[
    (df_hunt_full['n_rv_gaia_apogee_galah'] >= 2) & 
    (df_hunt_full['U_err'] < 10) & 
    (df_hunt_full['V_err'] < 10) & 
    (df_hunt_full['W_err'] < 10)
    ]
df_hunt_young = df_hunt_good.loc[df_hunt_good['age_myr'] < 15]
df_hunt_middle_aged = df_hunt_good.loc[df_hunt_good['age_myr'].between(15, 70)]
df_hunt_older = df_hunt_good.loc[df_hunt_good['age_myr'].between(0, 70)]
df_hunt_older.to_csv('/Users/cam/Downloads/hunt_catalog_0_70_myr.csv', index=False)

In [20]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']
cr135 = df_hunt_gagne[df_hunt_gagne['family'] == 'cr135']
ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']
gamma_vel = df_hunt_gagne[df_hunt_gagne['family'] == 'gamma_velorum']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]
m6 = m6.merge(df_hunt_partIII_to_merge, on='name', how='left')

sun = pd.DataFrame({'name' : ['Sun'], 'family' : ['Sun'], 'age_myr' : [8000], 'U' : [0.], 'V' : [0.], 'W' : [0.], 'x' : [0.], 'y' : [0.], 'z' : [0.], 'n_stars' : [1]})


In [21]:
# RW/Split/CS clusters
rw_region = pd.read_csv('/Users/cam/Downloads/rw_clusters.csv')
rw = rw_region.loc[rw_region.age_myr < 15]
remove = ['Theia_6', 'Teutsch_39', 'OCSN_32', 'Aveni-Hunter_1', 'IC_5146', 'Collinder_419', 'Dolidze_8', 'Theia_1920', 'Collinder_95', 'Mon_OB1-D', 'NGC_2264', 'UBC_159']
rw = rw[~rw['name'].isin(remove)]
rw = rw.rename(columns = column_renaming_dict)
#lacerta = ['HSC_661', 'vdBergh_152', 'HSC_705', 'OCSN_32', 'UBC_159', 'UPK_168', 'UPK_166', 'Teutch_39', 'Theia_100']

# The Split
split_region = pd.read_csv('/Users/cam/Downloads/the_split_selection.csv')
split = split_region.loc[split_region.age_myr < 15]
split = split.rename(columns = column_renaming_dict)

# Cepheus spur
cepheus_region = pd.read_csv('/Users/cam/Downloads/cepheus_spur_region.csv')
cepheus_spur = cepheus_region.loc[cepheus_region.age_myr < 30]
cepheus_spur = cepheus_spur.rename(columns = column_renaming_dict)


# new spur?
spur_clusters = ['Pismis_5', 'Collinder_197', 'FSR_1421', 'BH_56', 'HSC_2144', 'ASCC_79', 'UPK_604', 'FSR_1694', 'NGC_6178', 'OC_0684', 'HSC_2911', 'CWNU_170', 'NGC_6383']
spur = df_hunt_full.loc[df_hunt_full['name'].isin(spur_clusters)]
spur = spur.rename(columns = column_renaming_dict)

In [22]:
# new families
new_fam_1 = ['HSC_1373', 'OCSN_50', 'ASCC_24', 'HSC_1460', 'Theia_71', 'HSC_1340', 'CWNU_1057', 'CWNU_1111', 'CWNU_1092', 'HSC_1481', 'HSC_921', 'NGC_2232']
df_new_fam_1 = df_hunt_full.loc[df_hunt_full['name'].isin(new_fam_1)]

lacerta = ['HSC_661', 'vdBergh_152', 'HSC_705', 'OCSN_32', 'UBC_159', 'UPK_168', 'UPK_166', 'Teutch_39', 'Theia_100', 'HSC_699', 'CWNU_311', 'Teutsch_39', 'Aveni-Hunter_1', 'HSC_886']
df_lacerta = df_hunt_full.loc[df_hunt_full['name'].isin(lacerta)]

alphaPer_interloper = ['CWNU_1183', 'HSC_633', 'HSC_693', 'HSC_674', 'RSG_5', 'Theia_96', 'HSC_782', 'CWNU_519']
df_ap_interloper= df_hunt_full.loc[df_hunt_full['name'].isin(alphaPer_interloper)]


# Dark plot

In [23]:
# spurs
rw_data = StarClusterData(rw, data_name = 'Radcliffe Wave', min_size = 0, max_size = 9, color = 'white', opacity = 1., marker_style = 'circle')
split_data = StarClusterData(split, data_name = 'The Split', min_size = 0, max_size = 9, color = '#FFA07A', opacity = 1., marker_style = 'circle')
cepheus_spur_data = StarClusterData(cepheus_spur, data_name = 'Cepheus Spur', min_size = 0, max_size = 9, color = 'cyan', opacity = 1., marker_style = 'circle')
new_spur = StarClusterData(spur, data_name = 'Spur (?)', min_size = 0, max_size = 9, color = '#90EE90', opacity = 1., marker_style = 'circle')

# families
ap_data = StarClusterData(ap, data_name = '⍺Per', min_size = 0, max_size = 7, color = 'violet', opacity = 1., marker_style = 'circle')
m6_data = StarClusterData(m6, data_name = 'M6', min_size = 0, max_size = 7, color = 'cyan', opacity = 1., marker_style = 'circle')
cr135_data = StarClusterData(cr135, data_name = 'Cr135', min_size = 0, max_size = 7, color = 'orange', opacity = 1., marker_style = 'circle')
gamma_vel_data = StarClusterData(gamma_vel, data_name = '𝝲Vel', min_size = 0, max_size = 7, color = 'lime', opacity = 1., marker_style = 'circle')

# new families
unnamed_family_1= StarClusterData(df_new_fam_1, data_name = 'Unnamed family 1', min_size = 0, max_size = 7, color = '#90EE90', opacity = 1., marker_style = 'circle')
lacerta = StarClusterData(df_lacerta, data_name = 'Lacerta', min_size = 0, max_size = 7, color = 'coral', opacity = 1., marker_style = 'circle')
ap_interloper = StarClusterData(df_ap_interloper, data_name = '⍺Per interloper', min_size = 0, max_size = 7, color = '#191970', opacity = 1., marker_style = 'circle')

young_data = StarClusterData(df_hunt_young, data_name = 'Clusters (< 15 Myr)', min_size = 0., max_size = 7, color = 'black', opacity = 1., marker_style = 'circle')
older_data = StarClusterData(df_hunt_older, data_name = 'Clusters (< 70 Myr)', min_size = 0., max_size = 7, color = 'black', opacity = 1., marker_style = 'circle')
sun_data = StarClusterData(sun, data_name = 'Sun', min_size = 0, max_size = 7, color = 'yellow', opacity = 1., marker_style = 'circle')


sc = StarClusterCollection([young_data, older_data, sun_data, ap_data, m6_data, cr135_data, gamma_vel_data, rw_data, split_data, cepheus_spur_data, new_spur, unnamed_family_1, lacerta, ap_interloper])
grouping_dict = {
    'Cluster families' : ['⍺Per', 'M6', 'Cr135', '𝝲Vel', 'Lacerta', '⍺Per interloper', 'Unnamed family 1', 'Sun']
    # 'Dust structures' : ['Young clusters', 'Radcliffe Wave', 'The Split', 'Cepheus Spur', 'Spur (?)', 'Sun'],
}
#grouping_dict = None

In [24]:
xyz_widths = (1300, 1300, 300)
plot_3d = StarClusters3DPlotter(
    data_collection = sc, 
    xyz_widths = xyz_widths, 
    figure_theme = 'gray', 
    trace_grouping_dict=grouping_dict)

scatter_edenhofer['marker']['color'] = 'white'
scatter_edenhofer['marker']['opacity'] = 0.004

save_name = '/Users/cam/Downloads/gal_movie_sf_bursts.html'
#save_name = '/Users/cam/Downloads/gamma_vel_movie.html'
time_int = np.round(np.arange(0., -60.5, -0.5), 1)
#time_int = np.round(np.arange(0., -20.1, -0.1), 1)

fig3d_m6 = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = None, 
    static_traces_times = None, 
    static_traces_legendonly=False,
    focus_group = None,
    fade_in_time = 15, # Myr,
    fade_in_and_out = True
)

  0%|          | 0/208 [00:00<?, ?it/s]